<a href="https://colab.research.google.com/github/matheusccouto/meli-data-challenge-2021/blob/main/2_meli_data_challenge_2021_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![mercado-libre](https://ml-challenge.mercadolibre.com/static/images/logo-mercado-libre_en.png)

# MeLi Data Challenge 2021
# Model Development
On this notebook I analyze current and create new columns.

## The Challenge
Build a model to forecast item inventory days based on Mercado Libre historical data.

## The Task
The task is to predict how long it will take for the inventory of a certain item to be sold completely. In inventory management theory this concept is known as inventory days.

In the evaluation set I will be given the item target stock, and I will have to provide a prediction for the number of days it will take to run out. Possible values range from 1 to 30. Rather than giving a point estimate, you are expected to provide a score for each the possible outcomes.

To put it simply, you need to answer the following question:

**'What are the odds that the target stock will be sold out in one day?', 'What about in two days?' and so on until day 30.**

## Repository
This notebook is hosted on the this repository: [github.com/matheusccouto/meli-data-challenge-2021](https://github.com/matheusccouto/meli-data-challenge-2021)

Check-out the different branches to see all approaches tested.

## Load Data

### Google Drive
Processed data is stored on my personal google drive account.

In [ ]:
import os
from google.colab import drive

drive.mount("/gdrive")
base_dir = os.path.join("/gdrive", "My Drive", "Code", "meli-data-challenge-2021")
os.chdir(base_dir)

Mounted at /gdrive


### Train Data

In [ ]:
import pandas as pd

train_data = pd.read_parquet(os.path.join("data", "1-feature-engineering", "train_data.parquet"))
print(f"shape = {train_data.shape}")
train_data.head()

shape = (18091368, 35)


,sku,sold_quantity,current_price,listing_type_premium,shipping_payment_paid_shipping,shipping_logistic_type_cross_docking,shipping_logistic_type_drop_off,shipping_logistic_type_fulfillment,day,month_day,week_day,business_day,weekend,sold_quantity_month_mean,sold_quantity_month_std,price_month_mean,price_month_std,active,item_domain_id,item_domain_cluster,holiday,holiday_week,site_id,current_price_norm_domain_cluster,current_price_norm_domain_id,price_month_mean_norm_domain_cluster,price_month_mean_norm_domain_id,price_month_std_norm_domain_cluster,price_month_std_norm_domain_id,sold_quantity_norm_domain_cluster,sold_quantity_norm_domain_id,sold_quantity_month_mean_norm_domain_cluster,sold_quantity_month_mean_norm_domain_id,sold_quantity_month_std_norm_domain_cluster,sold_quantity_month_std_norm_domain_id
0,464801,0,156.779999,0,0,0,0,1,-59,1,0,1,0,0.344828,0.973795,171.811722,11.519452,1,2407,8,0,0,2,0.025961,0.350003,0.037537,0.474801,0.005893,0.229068,0.000000,0.000000,0.001059,0.014949,0.004106,0.03448
1,464801,0,156.779999,0,0,0,0,1,-58,2,1,1,0,0.344828,0.973795,171.811722,11.519452,1,2407,8,0,0,2,0.025961,0.350003,0.037537,0.474801,0.005893,0.229068,0.000000,0.000000,0.001059,0.014949,0.004106,0.03448
2,464801,0,156.779999,0,0,0,0,1,-57,3,2,1,0,0.344828,0.973795,171.811722,11.519452,1,2407,8,0,0,2,0.025961,0.350003,0.037537,0.474801,0.005893,0.229068,0.000000,0.000000,0.001059,0.014949,0.004106,0.03448
3,464801,0,156.779999,0,0,0,0,1,-56,4,3,1,0,0.344828,0.973795,171.811722,11.519452,1,2407,8,0,0,2,0.025961,0.350003,0.037537,0.474801,0.005893,0.229068,0.000000,0.000000,0.001059,0.014949,0.004106,0.03448
4,464801,1,156.779999,0,0,0,0,1,-55,5,4,1,0,0.344828,0.973795,171.811722,11.519452,1,2407,8,0,0,2,0.025961,0.350003,0.037537,0.474801,0.005893,0.229068,0.001028,0.009804,0.001059,0.014949,0.004106,0.03448


In [ ]:
train_data_dtypes = pd.read_json(os.path.join("data", "1-feature-engineering", "train_data_dtypes.json"), typ="series").to_dict()
train_data = train_data.astype(train_data_dtypes)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18091368 entries, 0 to 37616088
Data columns (total 35 columns):
 #   Column                                        Dtype  
---  ------                                        -----  
 0   sku                                           int32  
 1   sold_quantity                                 int16  
 2   current_price                                 float32
 3   listing_type_premium                          uint8  
 4   shipping_payment_paid_shipping                uint8  
 5   shipping_logistic_type_cross_docking          uint8  
 6   shipping_logistic_type_drop_off               uint8  
 7   shipping_logistic_type_fulfillment            uint8  
 8   day                                           int8   
 9   month_day                                     uint8  
 10  week_day                                      uint8  
 11  business_day                                  uint8  
 12  weekend                                       uint8  


### Validation Data

In [ ]:
valid_data = pd.read_parquet(os.path.join("data", "1-feature-engineering", "valid_data.parquet"))
print(f"shape = {valid_data.shape}")
valid_data.head()

shape = (19568911, 35)


,sku,sold_quantity,current_price,listing_type_premium,shipping_payment_paid_shipping,shipping_logistic_type_cross_docking,shipping_logistic_type_drop_off,shipping_logistic_type_fulfillment,day,month_day,week_day,business_day,weekend,sold_quantity_month_mean,sold_quantity_month_std,price_month_mean,price_month_std,active,item_domain_id,item_domain_cluster,holiday,holiday_week,site_id,current_price_norm_domain_cluster,current_price_norm_domain_id,price_month_mean_norm_domain_cluster,price_month_mean_norm_domain_id,price_month_std_norm_domain_cluster,price_month_std_norm_domain_id,sold_quantity_norm_domain_cluster,sold_quantity_norm_domain_id,sold_quantity_month_mean_norm_domain_cluster,sold_quantity_month_mean_norm_domain_id,sold_quantity_month_std_norm_domain_cluster,sold_quantity_month_std_norm_domain_id
29,464801,0,183.139999,0,0,0,0,1,-30,2,1,1,0,0.166667,0.461133,169.888336,5.507697,1,2407,8,0,0,2,0.030355,0.419309,0.037114,0.468581,0.002818,0.109522,0.0,0.0,0.000512,0.007225,0.001944,0.016328
30,464801,0,183.139999,0,0,0,0,1,-29,3,2,1,0,0.166667,0.461133,169.888336,5.507697,1,2407,8,0,0,2,0.030355,0.419309,0.037114,0.468581,0.002818,0.109522,0.0,0.0,0.000512,0.007225,0.001944,0.016328
31,464801,0,183.139999,0,0,0,0,1,-28,4,3,1,0,0.166667,0.461133,169.888336,5.507697,1,2407,8,0,0,2,0.030355,0.419309,0.037114,0.468581,0.002818,0.109522,0.0,0.0,0.000512,0.007225,0.001944,0.016328
32,464801,0,169.990005,0,0,0,0,1,-27,5,4,1,0,0.166667,0.461133,169.888336,5.507697,1,2407,8,0,0,2,0.028163,0.384735,0.037114,0.468581,0.002818,0.109522,0.0,0.0,0.000512,0.007225,0.001944,0.016328
33,464801,0,169.990005,0,0,0,0,1,-26,6,5,0,1,0.166667,0.461133,169.888336,5.507697,1,2407,8,0,0,2,0.028163,0.384735,0.037114,0.468581,0.002818,0.109522,0.0,0.0,0.000512,0.007225,0.001944,0.016328


In [ ]:
valid_data_dtypes = pd.read_json(os.path.join("data", "1-feature-engineering", "valid_data_dtypes.json"), typ="series").to_dict()
valid_data = valid_data.astype(valid_data_dtypes)
valid_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19568911 entries, 29 to 37660278
Data columns (total 35 columns):
 #   Column                                        Dtype  
---  ------                                        -----  
 0   sku                                           int32  
 1   sold_quantity                                 int16  
 2   current_price                                 float32
 3   listing_type_premium                          uint8  
 4   shipping_payment_paid_shipping                uint8  
 5   shipping_logistic_type_cross_docking          uint8  
 6   shipping_logistic_type_drop_off               uint8  
 7   shipping_logistic_type_fulfillment            uint8  
 8   day                                           int8   
 9   month_day                                     uint8  
 10  week_day                                      uint8  
 11  business_day                                  uint8  
 12  weekend                                       uint8  

### Test Data

In [ ]:
test_data = pd.read_parquet(os.path.join("data", "1-feature-engineering", "test_data.parquet")).reset_index()
print(f"shape = {test_data.shape}")
test_data.head()

shape = (16544160, 27)


,sku,target_stock,day,month_day,week_day,business_day,weekend,item_domain_id,item_domain_cluster,holiday,holiday_week,site_id,current_price,listing_type_premium,shipping_payment_paid_shipping,shipping_logistic_type_cross_docking,shipping_logistic_type_drop_off,shipping_logistic_type_fulfillment,price_month_mean,price_month_std,active,current_price_norm_domain_cluster,current_price_norm_domain_id,price_month_mean_norm_domain_cluster,price_month_mean_norm_domain_id,price_month_std_norm_domain_cluster,price_month_std_norm_domain_id
0,464801,3,0,1,3,1,0,2407,8,0,1,2,169.990005,0,0,0,0,1,169.888336,5.507697,1,0.028163,0.384735,0.037114,0.468581,0.002818,0.109522
1,464801,3,1,2,4,1,0,2407,8,1,1,2,169.990005,0,0,0,0,1,169.888336,5.507697,1,0.028163,0.384735,0.037114,0.468581,0.002818,0.109522
2,464801,3,2,3,5,0,1,2407,8,0,1,2,169.990005,0,0,0,0,1,169.888336,5.507697,1,0.028163,0.384735,0.037114,0.468581,0.002818,0.109522
3,464801,3,3,4,6,0,1,2407,8,1,1,2,169.990005,0,0,0,0,1,169.888336,5.507697,1,0.028163,0.384735,0.037114,0.468581,0.002818,0.109522
4,464801,3,4,5,0,1,0,2407,8,0,0,2,169.990005,0,0,0,0,1,169.888336,5.507697,1,0.028163,0.384735,0.037114,0.468581,0.002818,0.109522


In [ ]:
test_data_dtypes = pd.read_json(os.path.join("data", "1-feature-engineering", "test_data_dtypes.json"), typ="series").to_dict()
test_data = test_data.astype(test_data_dtypes)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16544160 entries, 0 to 16544159
Data columns (total 27 columns):
 #   Column                                Dtype  
---  ------                                -----  
 0   sku                                   uint64 
 1   target_stock                          uint16 
 2   day                                   uint8  
 3   month_day                             uint8  
 4   week_day                              uint8  
 5   business_day                          uint8  
 6   weekend                               uint8  
 7   item_domain_id                        uint16 
 8   item_domain_cluster                   uint8  
 9   holiday                               uint8  
 10  holiday_week                          uint8  
 11  site_id                               uint8  
 12  current_price                         float32
 13  listing_type_premium                  uint8  
 14  shipping_payment_paid_shipping        uint8  
 15  shipping_logi

## Cleaning
Drop some columns that I do not want to use now.

In [ ]:
to_drop = [
    'current_price',
    'sold_quantity_month_mean',
    'sold_quantity_month_std',
    'price_month_mean',
    'price_month_std',
    'item_domain_id',
    'item_domain_cluster',
    'site_id',
    'price_month_mean_norm_domain_cluster',
    'price_month_mean_norm_domain_id',
    'price_month_std_norm_domain_cluster',
    'price_month_std_norm_domain_id',
    'sold_quantity_norm_domain_cluster',
    'sold_quantity_norm_domain_id',
    'sold_quantity_month_mean_norm_domain_cluster',
    'sold_quantity_month_mean_norm_domain_id',
    'sold_quantity_month_std_norm_domain_cluster',
    'sold_quantity_month_std_norm_domain_id',
    'current_price_norm_domain_id',  # Has NAN
    ]

train_data = train_data[[col for col in train_data.columns if col not in to_drop]]
valid_data = valid_data[[col for col in valid_data.columns if col not in to_drop]]
test_data = test_data[[col for col in test_data.columns if col not in to_drop]]

Let's visualize the remaining cols.

In [ ]:
train_data.columns.tolist()

['sku',
 'sold_quantity',
 'listing_type_premium',
 'shipping_payment_paid_shipping',
 'shipping_logistic_type_cross_docking',
 'shipping_logistic_type_drop_off',
 'shipping_logistic_type_fulfillment',
 'day',
 'month_day',
 'week_day',
 'business_day',
 'weekend',
 'active',
 'holiday',
 'holiday_week',
 'current_price_norm_domain_cluster']

### Metric
Submissions will be scored using the **Ranked Probability Score (RPS)** metric.

**Ranked Probability Score** is a squared measure that compares the **estimated cumulative density function** of a probabilistic forecast with the **actual cumulative density function** of the corresponding observation. Under a discrete scenario of possible outcomes, the RPS formula is as follows:

$$ RPS = \frac{1}{N}\sum_{i=1}^{N}\left[\sum_{K=1}^{K}(Y_K(i) - O_K(i))^2\right] $$

where:

* $N$ is the number of rows in the dataset,
* $O$ represents the target,
* $K$ is the number of classes, and
* $Y$ is the predicted probability.

The **RPS** metric can be more easily understood graphically. The figure below illustrates how the value **RPS** takes is simply the square of the area where the predicted cumulative probability function and the ground truth do not overlap each other.

![rps-intuition](https://ml-challenge.mercadolibre.com/static/images/2021/metric_fig_5.png)

The main advantage of **RPS**, in comparison with other common metrics for forecasting problems, is that it is **sensitive to distance**, which means that a forecast is increasingly penalized the more its predictions differ from the actual outcome. This property is especially attractive for the task at hand from a business perspective: in order to manage inventory efficiently at MercadoLibre’s fulfillment centers, we would like the forecast to be as close as possible to the actual value of inventory days, besides the fact of just hitting the target number of inventory days with the highest point-probability estimate.

Disclaimer: you do NOT have to submit CUMULATIVE PROBABILITIES. **You are expected to provide point-probability estimates and the scorer itself will compute the accumulation.**

In [ ]:
import numpy as np


def rps(y, o):
    """Ranked Probability Score"""
    # Get intersection and transform into np arrays.
    intersection = y.join(o, how="inner", lsuffix="l", rsuffix="r").index
    y = y.loc[intersection].values
    o = o.loc[intersection].values

    # # Round Y.
    # y = np.round(y, decimals=4)

    # # Normalize Y.
    # y = np.divide(y, np.sum(y, axis=1).reshape(-1, 1))

    # Y and O are PDF and must be converted to CDF.
    y = np.cumsum(y, axis=1)
    o = np.cumsum(o, axis=1)

    # Sum the squares differences.
    return np.square(y - o).sum(axis=1).sum(axis=0) / y.shape[0]

## Train Model
Prepare data for training.

I will use the train stock from test data on valid data for training. As there are more SKUs on valid data than test data, some of them will be NaN.

In [ ]:
valid_stock = test_data.groupby("sku")["target_stock"].first()
valid_data = valid_data.join(valid_stock, on="sku")
valid_data = valid_data.fillna(valid_stock.mean())
valid_data["target_stock"] = valid_data["target_stock"].astype("uint16")
valid_data.head()

,sku,sold_quantity,listing_type_premium,shipping_payment_paid_shipping,shipping_logistic_type_cross_docking,shipping_logistic_type_drop_off,shipping_logistic_type_fulfillment,day,month_day,week_day,business_day,weekend,active,holiday,holiday_week,current_price_norm_domain_cluster,target_stock
29,464801,0,0,0,0,0,1,-30,2,1,1,0,1,0,0,0.030355,3
30,464801,0,0,0,0,0,1,-29,3,2,1,0,1,0,0,0.030355,3
31,464801,0,0,0,0,0,1,-28,4,3,1,0,1,0,0,0.030355,3
32,464801,0,0,0,0,0,1,-27,5,4,1,0,1,0,0,0.028163,3
33,464801,0,0,0,0,0,1,-26,6,5,0,1,1,0,0,0.028163,3


Create pivot data where 1 means that it ran out of stock and 0 not.

In [ ]:
def stockout(data):
    """Create stock-out pivot data."""
    stock = data.groupby("sku")["target_stock"].first()
    data = data.pivot(index="sku", columns="day", values="sold_quantity").fillna(0).cumsum(axis=1)
    stock = stock.clip(upper=data.max(axis=1))
    data = data.subtract(stock, axis=0)
    data = data >= 0
    data = data.cumsum(axis=1).cumsum(axis=1) == 1
    return data.astype("uint8")

valid_pivot = stockout(valid_data)
valid_pivot.head()

day,-30,-29,-28,-27,-26,-25,-24,-23,-22,-21,-20,-19,-18,-17,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1
sku,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
import os


def predict(estimator, train, test, save=True):
    """Make predictions."""
    # Do not try to predict days that were not even active.
    train = train[train["active"] == 1]
    test_inactive = test[test["active"] == 0][["sku", "day", "target_stock"]]
    test = test[test["active"] == 1]
    train = train.drop(columns="active")
    test = test.drop(columns="active")

    # Split data.
    X_train = train[[col for col in train.columns if col not in ["sku", "sold_quantity", "day", "target_stock"]]]
    X_test = test[[col for col in test.columns if col not in ["sku", "sold_quantity", "day", "target_stock"]]]
    y_train = train[["sold_quantity"]]
    y_stock = test.groupby("sku")["target_stock"].first()

    # Fit and predict.
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    y_pred = np.clip(y_pred, 0, None)  # Predictions cannot be less than 0.

    # Transform results in a DataFrame.
    y_pred = pd.DataFrame(y_pred, columns=["sold_quantity"], index=X_test.index)
    y_pred["sku"] = test["sku"]
    y_pred["day"] = test["day"]
    y_pred["target_stock"] = test["target_stock"]
    # Add innactive
    y_pred = y_pred.append(test_inactive)


    # Create pivot data (fill na days that do not exist).
    pred_pivot = stockout(y_pred)
    if save:
        os.makedirs("submissions", exist_ok=True)
        np.savetxt(os.path.join("submissions", f"{save}.csv.gz"), pred_pivot.values, fmt="%.4f", delimiter=",")
        # pd.DataFrame(pred_pivot.values).to_csv(os.path.join("submissions", f"{save}.csv.gz"), index=False, columns=False)
    
    return pred_pivot


def score(estimator, train, test):
    """Evaluate model."""
    pred_pivot = predict(estimator, train, test, save=False)
    test_pivot = stockout(test)
    return rps(pred_pivot, test_pivot)

In [ ]:
import lightgbm as lgb

estimator = lgb.LGBMRegressor()
score(estimator, train_data, valid_data)

7.409277427086044

In [ ]:
import datetime

pred = predict(estimator, train_data.append(valid_data), test_data, save=datetime.datetime.now())
pred.head()

day,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
sku,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
